In [172]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [176]:
house_dict = []

for page in range(15, 21):
    # Filter Out Coming Soon, Auction, Home Type: Apartment, Days On Zillow: 90 Days
    
    # Agency Listing
#     linked = 'https://www.zillow.com/ca/houses/' + str(page) + '_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-132.16065190625%2C%22east%22%3A-106.45264409375%2C%22south%22%3A29.712790718202044%2C%22north%22%3A44.40704031881662%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%2C%22category%22%3A%22cat1%22%2C%22pagination%22%3A%7B%22currentPage%22%3A' + str(page) + '%7D%7D'
    
    # Other Listing
    linked = 'https://www.zillow.com/ca/houses/' + str(page) + '_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-132.16065190625%2C%22east%22%3A-106.45264409375%2C%22south%22%3A29.712790718202044%2C%22north%22%3A44.40704031881662%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%2C%22category%22%3A%22cat2%22%2C%22pagination%22%3A%7B%22currentPage%22%3A' + str(page) + '%7D%7D'

    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    print('page: ', page)
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
#             listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
            if (len(loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")) < 3):
                zestimate = 'null'
                rent_zestimate = 'null'
            else:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[0].text
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[2].text
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent zestimate', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
#                 "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(5)
            
        except AttributeError as e:
            continue

# print(house_dict)

page:  15
https://www.zillow.com/homedetails/414625-Scott-Rd-Doyle-CA-96109/2069671319_zpid/
zestimate $349,700
rent zestimate $1,964/mo
https://www.zillow.com/homedetails/4053-Summer-Ct-Merced-CA-95340/61486727_zpid/
zestimate $542,700
rent zestimate $2,528/mo
https://www.zillow.com/homedetails/5347-Holly-Ridge-Dr-Camarillo-CA-93012/16373733_zpid/
zestimate $775,833
rent zestimate $3,249/mo
https://www.zillow.com/homedetails/31472-Mohawk-Rd-Coarsegold-CA-93614/19142011_zpid/
zestimate $343,100
rent zestimate $2,400/mo
https://www.zillow.com/homedetails/916-S-Farragut-St-Ridgecrest-CA-93555/249712315_zpid/
zestimate $313,400
rent zestimate $1,814/mo
https://www.zillow.com/homedetails/22507-Robin-Hood-Dr-Twain-Harte-CA-95383/242152638_zpid/
zestimate $288,800
rent zestimate $1,294/mo
https://www.zillow.com/homedetails/29823-Old-Sycamore-Ln-Murrieta-CA-92563/52477167_zpid/
zestimate $631,600
rent zestimate $2,999/mo
https://www.zillow.com/homedetails/1517-C-St-Fresno-CA-93706/174929141_z

In [177]:
house_data = pd.DataFrame(house_dict)
house_data

,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,"$600,000","414625 Scott Rd, Doyle, CA 96109",414625 Scott Rd,Doyle,CA,96109,3,2,"1,660",20 Acres,871200,SingleFamily,1984,0 spaces,null,"$349,700","$1,964/mo",$361,https://www.zillow.com/homedetails/414625-Scot...
1,"$610,000","4053 Summer Ct, Merced, CA 95340",4053 Summer Ct,Merced,CA,95340,5,3,"2,899","9,583 sqft",9583,SingleFamily,2004,3 Covered Parking spaces,null,"$542,700","$2,528/mo",$210,https://www.zillow.com/homedetails/4053-Summer...
2,"$760,000","5347 Holly Ridge Dr, Camarillo, CA 93012",5347 Holly Ridge Dr,Camarillo,CA,93012,4,2,"1,636","6,098 sqft",6098,SingleFamily,1975,2 Parking spaces,null,"$775,833","$3,249/mo",$465,https://www.zillow.com/homedetails/5347-Holly-...
3,"$345,000","31472 Mohawk Rd, Coarsegold, CA 93614",31472 Mohawk Rd,Coarsegold,CA,93614,3,2,"1,680",1 Acres,43560,SingleFamily,1989,5 Parking spaces,null,"$343,100","$2,400/mo",$205,https://www.zillow.com/homedetails/31472-Mohaw...
4,"$325,000","916 S Farragut St, Ridgecrest, CA 93555",916 S Farragut St,Ridgecrest,CA,93555,4,2,"1,847","6,098 sqft",6098,SingleFamily,2016,Garage - Attached,null,"$313,400","$1,814/mo",$176,https://www.zillow.com/homedetails/916-S-Farra...
5,"$300,000","22507 Robin Hood Dr, Twain Harte, CA 95383",22507 Robin Hood Dr,TwainHarte,CA,95383,2,2,940,0.45 Acres,19602,SingleFamily,1988,2 Parking spaces,null,"$288,800","$1,294/mo",$319,https://www.zillow.com/homedetails/22507-Robin...
6,"$640,000","29823 Old Sycamore Ln, Murrieta, CA 92563",29823 Old Sycamore Ln,Murrieta,CA,92563,3,3,"2,125","6,899 sqft",6899,SingleFamily,2000,2 Parking spaces,null,"$631,600","$2,999/mo",$301,https://www.zillow.com/homedetails/29823-Old-S...
7,"$300,000","1517 C St, Fresno, CA 93706",1517 C St,Fresno,CA,93706,5,2,"1,589","5,227 sqft",5227,SingleFamily,2005,Garage - Attached,null,"$239,000","$1,849/mo",$189,https://www.zillow.com/homedetails/1517-C-St-F...
8,"$377,000","340 Trinity Cir, Hanford, CA 93230",340 Trinity Cir,Hanford,CA,93230,3,2,"1,778","10,018 sqft",10018,SingleFamily,1986,Garage - Attached,null,"$347,900","$1,864/mo",$212,https://www.zillow.com/homedetails/340-Trinity...
9,"Est. $1,886,000","4490 Arcadia Ave, Oakland, CA 94602",4490 Arcadia Ave,Oakland,CA,94602,5,4,"2,866",0.32 Acres,13939.2,SingleFamily,1968,Garage - Attached,null,null,null,null,https://www.zillow.com/homedetails/4490-Arcadi...


In [178]:
# Export file as a CSV, without the Pandas index, but with the header
# house_data.to_csv("Output/Zillow/Q2-2021_Single_Family_AgentListing_page15-20.csv", index=False, header=True)
house_data.to_csv("Output/Zillow/Q2-2021_Single_Family_OtherListing_page15-20.csv", index=False, header=True)